In [1]:
%matplotlib inline
import os, sys, time
import pandas as pd
import numpy as np
import seaborn as sb
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import csr_matrix, hstack


import matplotlib.pyplot as plt
import cPickle as pickle

pickle_column_names_filee = data_path + 'cat_col_names.pkl'

In [2]:
data_path = '/home/ymm/bosch/'

train_num_file   = 'train_numeric.csv'
train_cat_file   = 'train_categorical.csv'
train_date_file  = 'train_date.csv'
test_num_file    = 'test_numeric.csv'
test_cat_file    = 'test_categorical.csv'
test_date_file   = 'test_date.csv'

sample_submission_file   = 'sample_submission.csv'

### this section is only necesary to run for the first time

In [15]:
## section to collect the column names so that a pickle file of column names can be created
tmp_train_cat = pd.read_csv(data_path + train_cat_file, index_col='Id', nrows=10)

'''
for col, dtype in zip(tmp_train_cat.columns, tmp_train_cat.dtypes):
    print len(train_cat[col].unique()), dtype
'''

## save the column names to pickle file
col_names = tmp_train_cat.columns.tolist()
with open(pickle_column_names_filee, 'wb') as pickle_file:
    pickle.dump(col_names, pickle_file)

In [7]:
## section to load column names
with open(pickle_column_names_filee, 'rb') as pickle_file:
    cat_column_names = pickle.load(pickle_file)
    
column_types = [np.object] * len(cat_column_names)
column_types_dict = dict(zip(cat_column_names, column_types))

### section to load categorical train/test data

In [10]:
% time test_cat = pd.read_csv(data_path + test_cat_file, index_col='Id', dtype=column_types_dict)

CPU times: user 6min 3s, sys: 26.2 s, total: 6min 29s
Wall time: 6min 46s


In [11]:
% time train_cat = pd.read_csv(data_path + train_cat_file, index_col='Id', dtype=column_types_dict)

CPU times: user 6min 24s, sys: 13.2 s, total: 6min 37s
Wall time: 6min 42s


In [16]:
print train_cat.shape, test_cat.shape

(1183747, 2140) (1183748, 2140)


In [17]:
train_cat.head()

,L0_S1_F25,L0_S1_F27,L0_S1_F29,L0_S1_F31,L0_S2_F33,L0_S2_F35,L0_S2_F37,L0_S2_F39,L0_S2_F41,L0_S2_F43,...,L3_S49_F4225,L3_S49_F4227,L3_S49_F4229,L3_S49_F4230,L3_S49_F4232,L3_S49_F4234,L3_S49_F4235,L3_S49_F4237,L3_S49_F4239,L3_S49_F4240
Id,,,,,,,,,,,,,,,,,,,,,
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
train_ids = train_cat.index.tolist()
test_ids = test_cat.index.tolist()
print len(set(train_ids).intersection(test_ids)), len(set(test_ids).intersection(train_ids))

0 0


In [27]:
full_cat_data = pd.concat([train_cat, test_cat])

In [29]:
full_cat_data.head()

,L0_S1_F25,L0_S1_F27,L0_S1_F29,L0_S1_F31,L0_S2_F33,L0_S2_F35,L0_S2_F37,L0_S2_F39,L0_S2_F41,L0_S2_F43,...,L3_S49_F4225,L3_S49_F4227,L3_S49_F4229,L3_S49_F4230,L3_S49_F4232,L3_S49_F4234,L3_S49_F4235,L3_S49_F4237,L3_S49_F4239,L3_S49_F4240
Id,,,,,,,,,,,,,,,,,,,,,
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
del train_cat, test_cat

In [32]:
import gc
gc.collect()

4147

In [ ]:
def Encode_Categorical_Columns(df, fill_missing = False):
    le = LabelEncoder()
    if fill_missing:
        df = df.fillna(value='missing')
    for col, dtype in zip(df.columns, df.dtypes):
        if dtype == 'object':
            df[col] = le.fit_transform(df[col])
    return df

encoded_full_cat_data = Encode_Categorical_Columns(full_cat_data, True)